# Precificação de ativos com CAPM (Capital Asset Pricing Model)

In [2]:
import pandas as pd
import numpy as np
import statistics
import math
from scipy import stats
import seaborn as srn
import matplotlib.pyplot as plt
import plotly.express as px
srn.set()

## CAPM

Descreve as relações entre o retorno esperado e o risco, comparando o portfólio com o mercado (IBOVESPA).

$$
Ri = Rf + \beta i * (Rm - Rf)
$$

- $Rm$: retorno esperado do portfólio
- $Rf$: retorno sem risco
- $\beta$: comparação entre a carteira e o mercado
    - $=1$: fortemente relacionado
    - $<1$ (defensivo): menos volátil que o mercado
    - $>1$ (agressivo): mais volátil que o mercado


In [4]:
dataset = pd.read_csv('acoes.csv')
dataset.drop('Date', axis=1, inplace=True)
print(dataset.shape)
dataset.head()

(1798, 6)


,GOL,CVC,WEGE,MGLU,TOTS,BOVA
0,14.99,15.20,5.923076,0.232812,11.910702,47.259998
1,14.85,15.00,5.963461,0.237187,11.544731,46.320000
2,15.21,14.80,5.875000,0.234062,10.822770,46.580002
3,14.55,14.67,5.807692,0.241875,10.746248,48.150002
4,14.27,14.15,5.905769,0.240000,10.995774,48.509998


In [7]:
dataset_normalizado = dataset.copy()
for i in dataset.columns:
    dataset_normalizado[i] = dataset[i]/dataset[i][0]
dataset_normalizado.head()

,GOL,CVC,WEGE,MGLU,TOTS,BOVA
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.990660,0.986842,1.006818,1.018792,0.969274,0.980110
2,1.014676,0.973684,0.991883,1.005369,0.908659,0.985612
3,0.970647,0.965132,0.980520,1.038928,0.902235,1.018832
4,0.951968,0.930921,0.997078,1.030875,0.923184,1.026449


In [10]:
taxa_retorno = (dataset_normalizado/dataset_normalizado.shift(1)) - 1
taxa_retorno.fillna(0, inplace=True)
taxa_retorno.head()

,GOL,CVC,WEGE,MGLU,TOTS,BOVA
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.009340,-0.013158,0.006818,0.018792,-0.030726,-0.019890
2,0.024242,-0.013333,-0.014834,-0.013175,-0.062536,0.005613
3,-0.043392,-0.008784,-0.011457,0.033380,-0.007070,0.033705
4,-0.019244,-0.035447,0.016887,-0.007752,0.023220,0.007477


In [11]:
# taxa de retorno anual
taxa_retorno.mean() * 246

GOL     0.309629
CVC     0.174885
WEGE    0.299592
MGLU    0.670243
TOTS    0.224929
BOVA    0.156672
dtype: float64

## Cálculo do $\beta$ 

### Regressão Linear

In [16]:
figura = px.scatter(taxa_retorno, x='BOVA', y='MGLU', title='BOVA x MAGALU')
figura.show()

In [17]:
beta, alpha = np.polyfit(x=taxa_retorno['BOVA'], y=taxa_retorno['MGLU'], deg=1)
print(f'beta:{beta}')
print(f'alpha:{alpha}')

beta:1.1332314895402156
alpha:0.002002837069247779


In [18]:
figura = px.scatter(taxa_retorno, x='BOVA', y='MGLU', title='BOVA x MGLU')
figura.add_scatter(x=taxa_retorno['BOVA'], y=beta*taxa_retorno['BOVA']+alpha)
figura.show()

### Covariância e variância

In [24]:
matriz_cov = taxa_retorno[['MGLU', 'BOVA']].cov() * 246
matriz_cov

,MGLU,BOVA
MGLU,0.417839,0.076583
BOVA,0.076583,0.067579


In [25]:
cov_mglu_bova = matriz_cov.iloc[0, 1]
cov_mglu_bova

0.07658271081688324

In [27]:
variancia_bova = taxa_retorno['BOVA'].var() * 246
variancia_bova

0.06757905293291408

In [29]:
beta_mglu = cov_mglu_bova / variancia_bova
beta_mglu

1.1332314895402147